In [13]:
import json_lines
import re
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import string
import pandas as pd
import mtranslate

In [14]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [48]:
df = pd.DataFrame(columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()

#For some reason, please returns positive sentiment because it assumes 'pleasing' but normally people just asking politely
weird_sentiment_words = ['la', 'please']

file = json.load(open('fitnessfirst6months.json', encoding="utf8"))
for line in file['data']:
    
    try:
        #Check if key exists
        line['comments']
        #print(line['message'])
        
        for comments in line['comments']['data']:
            print("New comment")
            print('---------------------------------------')
            sentence = comments['message']
            
            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            #print(sentences)
            
            #Sentiment for each sentence
            for s in sentences:
                
                s = s.lower()
                s = s.replace('fitness first', ' ')
                s = re.sub(r"\byin\b", "", s)
                s = re.sub(r"\byang\b", "", s)
                
                print("Sentence: ", s)
                sentiment = 0.0
                #print(s)
                
                tokenized = word_tokenize(s)
                #print(tokenized)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized
                
                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)
                
                previous_word = ""
                list_of_nouns = []
                
                #Get the sentiment for each word
                for w, t in tagged:
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel', 'cancellation']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel', 'cancellation']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word
                    
                    #Extract nouns
                    if t.startswith('N') or w == 'post':
                        if w != 'i' and w != 'la' and w != 'hi':
                            list_of_nouns.append(w)
                        
                try:
                    df2 = pd.DataFrame([[line['message'], s, ','.join(list_of_nouns), sentiment, 'Facebook Comment Fitness First']],columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])
                except Exception:
                    df2 = pd.DataFrame([['No message', s, ','.join(list_of_nouns), sentiment, 'Facebook Comment Fitness First']],columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])
                    
                df = pd.concat([df2,df], ignore_index=True)
                    
                # sum greater than 0 => positive sentiment
                if sentiment >= 0:
                    print("This sentence has a positive sentiment: " + str(sentiment))
                elif sentiment < 0:
                    print("This sentence has a negative sentiment: " + str(sentiment))
        print()
    except:
        continue

New comment
---------------------------------------
Sentence:  farah zuhanazullkefle pnjm card nk msk ni..
[('farah', 'NN'), ('zuhanazullkefle', 'NN'), ('pnjm', 'FW'), ('card', 'NN'), ('nk', 'NN'), ('msk', 'NN'), ('ni..', 'NN')]
Lemmatized word:  farah
Synset not found
Lemmatized word:  zuhanazullkefle
Synset not found
Lemmatized word:  pnjm
Synset not found
Lemmatized word:  card
card  |  Synset('card.n.01')  |  <card.n.01: PosScore=0.0 NegScore=0.0>  |  one of a set of small pieces of stiff paper marked in various ways and used for playing games or for telling fortunes
Lemmatized word:  nk
Synset not found
Lemmatized word:  msk
Synset not found
Lemmatized word:  ni..
Synset not found
This sentence has a positive sentiment: 0.0
New comment
---------------------------------------
Sentence:  ben lucas your bags
[('ben', 'NN'), ('lucas', 'VBZ'), ('your', 'PRP$'), ('bags', 'NNS')]
Lemmatized word:  ben
ben  |  Synset('ben.n.01')  |  <ben.n.01: PosScore=0.125 NegScore=0.0>  |  a mountain o

In [49]:
#df.to_json('chi_fitness_comments_sentiment.json')

In [50]:
df_filtered = df[df['Sentiment'] != 0]
print(len(df_filtered[df_filtered['Sentiment'] < 0]))
print(len(df_filtered[df_filtered['Sentiment'] > 0]))

98
143


In [52]:
df_filtered[df_filtered['Sentiment'] > 0]

,Post,Comment,Topic,Sentiment,Source
3,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",the best of this,,0.750,Facebook Comment Fitness First
4,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",respect and love it !!,respect,0.500,Facebook Comment Fitness First
6,"""I AM a fighter not a quitter"", C. K. Loh, Fri...","you just inspired me, awesome",,0.125,Facebook Comment Fitness First
15,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",what is the cost like?,cost,0.250,Facebook Comment Fitness First
16,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",how do i enrol if i am interested in this prog...,program,0.500,Facebook Comment Fitness First
18,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",real spartan training !,training,0.125,Facebook Comment Fitness First
19,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",sakthidasan subaramonia check it out 💪,,0.500,Facebook Comment Fitness First
24,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",he's one of the passionate and happiest men i'...,"men,cool",0.500,Facebook Comment Fitness First
26,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",someone who helps you put life into a better p...,"someone,life,perspective",0.375,Facebook Comment Fitness First
27,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",you know how sometimes you meet someone who in...,"someone,words",0.125,Facebook Comment Fitness First


In [53]:
df_filtered[df_filtered['Sentiment'] < 0]

,Post,Comment,Topic,Sentiment,Source
8,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",winter hardbrick what is ur excuse,"winter,excuse",-0.125,Facebook Comment Fitness First
13,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",winner and fighter will never lose or quit .,"winner,fighter",-1.000,Facebook Comment Fitness First
14,"""I AM a fighter not a quitter"", C. K. Loh, Fri...","dear friend, look at this guy, he is so determ...","friend,guy,arm",-0.125,Facebook Comment Fitness First
25,"""I AM a fighter not a quitter"", C. K. Loh, Fri...",well it just happened to me.. met this tough ...,"men,life,defect",-0.625,Facebook Comment Fitness First
32,Get dressed down in swanky summer gear and joi...,is this open for public?,,-0.375,Facebook Comment Fitness First
36,Please be advised the main elevators to Fitnes...,so long and not rectified yet 😇😇,,-0.375,Facebook Comment Fitness First
42,Please be advised the main elevators to Fitnes...,fit people should not use elevator,"people,elevator",-0.125,Facebook Comment Fitness First
44,Please be advised the main elevators to Fitnes...,just that this time is worse.,time,-0.500,Facebook Comment Fitness First
46,Please be advised the main elevators to Fitnes...,cannot blame ff entirely.,,-0.375,Facebook Comment Fitness First
53,Early Bird Tickets for Fit in The City are now...,"i did ask front desk for my membership number,...","front,desk,membership,number,work",-0.500,Facebook Comment Fitness First


In [56]:
df_filtered.to_json('fitnessfirstcommentsentiment.json', orient='records')

In [57]:
test = df_filtered[df_filtered['Sentiment'] > 0]
test[test['Topic'].str.contains('obstacles')]

,Post,Comment,Topic,Sentiment,Source
220,11 obstacle challenges for you to complete at ...,irene puffer wong nah the obstacles,"puffer,obstacles",0.250,Facebook Comment Fitness First
274,The pain you feel today will be your strength ...,it was definitely challenging as there were 11...,"obstacles,team",0.375,Facebook Comment Fitness First
